# Model development

In [1]:
from src.data.data_loader import *

# Load and preprocess the data
dataset_loader = DatasetLoader(dataset_name='ibm/MedMentions-ZS', source='huggingface', model_name='UFNLP/gatortron-base')
dataset, classmap, umls_label_code, _ = dataset_loader.load_dataset()

# Remove columns not used in training
dataset = dataset.remove_columns(['tokens', 'ner_tags', 'token_labels'])

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and preprocessing the dataset ...


In [2]:
from src.models.model import *

# Create an NER model from a base pretrained model
model_loader = ModelLoader('UFNLP/gatortrons')
NER_model, tokenizer, config = model_loader.load_model(num_labels = len(classmap.names))

The device to run the model: cuda
Load the pretrained model ...


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MegatronBertForTokenClassification were not initialized from the model checkpoint at UFNLP/gatortrons and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model has 354.262059millions parameters.


In [3]:
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
from seqeval.metrics import f1_score

# Create a collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define metrics
def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)

    return {"f1": f1_score(y_true, y_pred)}
                                                   
# Specify training args
epoch = 3
batch_size = 16
logging_steps = len(dataset["train"]) // batch_size
training_args = TrainingArguments(
                    output_dir="../models/medmentions/gatortrons/",
                    num_train_epochs=epoch,
                    per_device_train_batch_size=batch_size,
                    per_device_eval_batch_size=batch_size,
                    load_best_model_at_end=True,
                    weight_decay = 0.01,
                    logging_steps=logging_steps,
                    log_level='info',
                    evaluation_strategy='epoch',
                    eval_steps=50,
                    save_strategy='epoch',
                    disable_tqdm=False
                )

# Create a trainer                                           
trainer = Trainer(model=NER_model, 
                  args=training_args, 
                  data_collator=data_collator, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset["train"],
                  eval_dataset=dataset["validation"], 
                  tokenizer=tokenizer)

# Train the model
trainer.train()

***** Running training *****
  Num examples = 26,770
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5,022
  Number of trainable parameters = 354,262,059


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 7.06 MiB is free. Including non-PyTorch memory, this process has 14.74 GiB memory in use. Of the allocated memory 14.32 GiB is allocated by PyTorch, and 293.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF